In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Dataset_Skripsi_Processed.csv'  # Path absolut
df = pd.read_csv(file_path)

In [3]:
# Menampilkan data pertama untuk memastikan dataset terbaca dengan benar
pd.set_option('display.max_rows', None)
print(df)

    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
0             P01         G01  0.0  0.0        0.0       0.0
1             P01         G02  0.6  1.0        0.8       0.8
2             P01         G03  0.0  0.0        0.0       0.0
3             P01         G04  0.0  0.0        0.0       0.0
4             P01         G05  0.0  0.0        0.0       0.0
5             P01         G06  0.0  0.0        0.0       0.0
6             P01         G07  0.0  0.0        0.0       0.0
7             P01         G08  0.0  0.0        0.0       0.0
8             P01         G09  1.0  0.8        0.9       1.0
9             P01         G10  0.8  1.0        0.9       1.0
10            P01         G11  0.6  1.0        0.8       0.8
11            P01         G12  0.6  1.0        0.8       0.8
12            P01         G13  0.0  0.0        0.0       0.0
13            P01         G14  0.0  0.0        0.0       0.0
14            P01         G15  0.0  0.0        0.0       0.0
15            P01       

In [4]:
# Filter data untuk P08 saja
df_P08 = df[df['kode_penyakit'] == 'P08']  # Ambil data hanya untuk P08

In [5]:
# Menampilkan data yang diambil untuk P08
print("Data untuk P08:")
print(df_P08)

Data untuk P08:
    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
217           P08         G01  0.6  1.0        0.8       0.8
218           P08         G02  0.0  0.0        0.0       0.0
219           P08         G03  0.0  0.0        0.0       0.0
220           P08         G04  0.6  1.0        0.8       0.8
221           P08         G05  0.0  0.0        0.0       0.0
222           P08         G06  0.6  0.8        0.7       0.8
223           P08         G07  0.0  0.0        0.0       0.0
224           P08         G08  0.0  0.0        0.0       0.0
225           P08         G09  0.0  0.0        0.0       0.0
226           P08         G10  0.0  0.0        0.0       0.0
227           P08         G11  0.0  0.0        0.0       0.0
228           P08         G12  0.0  0.0        0.0       0.0
229           P08         G13  0.0  0.0        0.0       0.0
230           P08         G14  0.0  0.0        0.0       0.0
231           P08         G15  0.0  0.0        0.0       0.0
232     

In [6]:
# Set seed for reproducibility
np.random.seed(42)

## Inisialisasi Parameter

In [7]:
# Parameter PSO tetap
w_max = 0.9  
w_min = 0.4  
n_particles = 10  # Jumlah partikel
n_iterations = 10  # Jumlah iterasi

## Fungsi Fitness

In [8]:
# Fungsi fitness untuk mengukur sejauh mana posisi partikel mendekati cf_pakar
def fitness(position, cf_pakar):
    return np.mean((cf_pakar - position) ** 2, axis=1)  # MSE 

## Implementasi

In [9]:
# Fungsi untuk menjalankan PSO dengan parameter bobot inersia yang berbeda untuk setiap penyakit
def run_pso(cf_pakar, c1, c2):
    fitness_best_results = []
    
    for trial in range(1, 11):  # 10 uji coba
        # Inisialisasi kecepatan dan posisi partikel secara acak
        position = np.random.uniform(0.0, 1.0, (n_particles, len(cf_pakar)))  # Posisi partikel
        velocity = np.random.uniform(-0.1, 0.1, (n_particles, len(cf_pakar)))  # Kecepatan partikel

        # Inisialisasi pbest dan gbest
        pbest = position.copy()  # Set pbest awal sama dengan posisi partikel
        pbest_score = np.array([np.mean(np.abs(cf_pakar - p)) for p in position])  # Nilai fitness awal untuk pbest
        gbest = pbest[np.argmin(pbest_score)]  # gbest adalah posisi dengan nilai fitness terbaik
        gbest_score = np.min(pbest_score)  # Nilai fitness terbaik untuk gbest

        # Iterasi PSO
        for iteration in range(n_iterations):
            w = w_max - (w_max - w_min) * (iteration / n_iterations)  # Interpolasi w antara w_max dan w_min

            # Update posisi dan kecepatan setiap partikel
            for i in range(n_particles):
                r1 = np.random.rand(len(cf_pakar))  # Random number untuk C1
                r2 = np.random.rand(len(cf_pakar))  # Random number untuk C2

                # Update kecepatan
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - position[i]) + c2 * r2 * (gbest - position[i])

                # Update posisi
                position[i] += velocity[i]

                # Pastikan posisi berada dalam rentang 0 dan 1
                position[i] = np.clip(position[i], 0.0, 1.0)

            # Evaluasi fitness setiap partikel
            score = fitness(position, cf_pakar)

            # Update pbest dan gbest
            for i in range(n_particles):
                if score[i] < pbest_score[i]:
                    pbest[i] = position[i]  # Update pbest dengan posisi partikel yang lebih baik
                    pbest_score[i] = score[i]

            # Update gbest jika ditemukan pbest yang lebih baik
            gbest_score_new = np.min(pbest_score)
            if gbest_score_new < gbest_score:
                gbest = pbest[np.argmin(pbest_score)]  # Update gbest dengan posisi pbest terbaik
                gbest_score = gbest_score_new

        # Simpan hasil fitness terbaik untuk setiap uji coba
        fitness_best_results.append(gbest_score)

    # Mengembalikan hasil fitness terbaik untuk 10 uji coba
    return fitness_best_results

In [10]:
# Daftar penyakit yang ingin diuji (hanya P08 untuk sekarang)
penyakit_codes = ['P08']  # Fokus pada P08 saja

In [11]:
# Menyimpan hasil uji coba per penyakit
results = []

In [12]:
# Kombinasi bobot inersia yang diuji
c1_values = [1.0, 1.5, 0.2]
c2_values = [1.0, 1.5, 0.2]

In [13]:
# Fokus pada P08
for penyakit in penyakit_codes:
    cf_pakar = df[df['kode_penyakit'] == penyakit]['cf_pakar'].values
    for c1 in c1_values:
        for c2 in c2_values:
            # Jalankan PSO dan simpan hasil per percobaan
            fitness_results = run_pso(cf_pakar, c1, c2)
            
            # Simpan hasil per percobaan (trial) untuk c1 dan c2 yang diuji
            for trial_num, fitness_score in enumerate(fitness_results, 1):
                results.append({
                    'Penyakit': penyakit,
                    'c1': c1,
                    'c2': c2,
                    'Percobaan': trial_num,
                    'Fitness Terbaik': fitness_score
                })

In [14]:
# Membuat DataFrame untuk menampilkan hasil
df_results = pd.DataFrame(results)

pd.set_option('display.max_rows', 1000)

In [15]:
# Menampilkan hasil tabel per percobaan
print(df_results)

   Penyakit   c1   c2  Percobaan  Fitness Terbaik
0       P08  1.0  1.0          1         0.090691
1       P08  1.0  1.0          2         0.088651
2       P08  1.0  1.0          3         0.132668
3       P08  1.0  1.0          4         0.148855
4       P08  1.0  1.0          5         0.133863
5       P08  1.0  1.0          6         0.144972
6       P08  1.0  1.0          7         0.085921
7       P08  1.0  1.0          8         0.123526
8       P08  1.0  1.0          9         0.113604
9       P08  1.0  1.0         10         0.113145
10      P08  1.0  1.5          1         0.119001
11      P08  1.0  1.5          2         0.122826
12      P08  1.0  1.5          3         0.063400
13      P08  1.0  1.5          4         0.098224
14      P08  1.0  1.5          5         0.124892
15      P08  1.0  1.5          6         0.071780
16      P08  1.0  1.5          7         0.132067
17      P08  1.0  1.5          8         0.104248
18      P08  1.0  1.5          9         0.087964


In [16]:
# Menghitung rata-rata fitness terbaik untuk setiap kombinasi ωmax dan ωmin
avg_fitness_results = df_results.groupby(['Penyakit', 'c1', 'c2'])['Fitness Terbaik'].mean().reset_index()

In [17]:
# Menampilkan kombinasi terbaik per penyakit berdasarkan rata-rata fitness
print("\nKombinasi c1 dan c2 terbaik per penyakit:")
print(avg_fitness_results)


Kombinasi c1 dan c2 terbaik per penyakit:
  Penyakit   c1   c2  Fitness Terbaik
0      P08  0.2  0.2         0.168489
1      P08  0.2  1.0         0.142357
2      P08  0.2  1.5         0.136976
3      P08  1.0  0.2         0.173453
4      P08  1.0  1.0         0.117590
5      P08  1.0  1.5         0.105944
6      P08  1.5  0.2         0.179717
7      P08  1.5  1.0         0.121056
8      P08  1.5  1.5         0.110590


In [18]:
# Jika ingin menambahkan header dan data ke file yang sudah ada (append):
df_results.to_csv('pengujian_koefisien_akselerasi_P08.csv', mode='a', header=False, index=False)
